# Notebook for predicting motor impairment severity using functional connectivity data

In this notebook, we train a simple MLP to predict motor impairment using either subnetworks or individual connections from fMRI data.

## Imports

In [1]:
from utils.data_loading import load_clinical_data, load_fmri_data, select_subnetwork
from utils.model import train_eval
import numpy as np
from scipy.stats import wilcoxon
import warnings

warnings.filterwarnings("ignore")

## Directories

In [2]:
clinical_data_path = '../data/private_data/clinical_data.csv'
gait_scores_path = '../data/private_data/gait_scores.csv'
fc_matrices_dir = '../data/private_data/fc_matrices/'

## Predict motor impairment from FC (rs-fMRI) data

We use leave-one-out validation and average results across 20 runs.

In [3]:
clinical_data = load_clinical_data(clinical_data_path, gait_scores_path)
fmri_data = load_fmri_data(fc_matrices_dir)
networks = ['whole-brain','pallidal-sensorimotor', 'cerebellar-motor', 'cerebellar-prefrontal', 'top-cerebellar-prefrontal', 'top-whole-brain']
for network in networks:
    use_random_connections = False
    if network == 'whole-brain':
        network_data = fmri_data
    else:
        network_data = select_subnetwork(fmri_data, clinical_data, network)
    
    X = []
    y = []
    for _, row in clinical_data.iterrows():
        X.append(network_data[row['PID']])
        y.append(row['majority vote'] - 1)
    X_np = np.array(X)
    y_np = np.array(y)
    auc_scores, f1_scores, precision_scores, recall_scores = train_eval(X_np, y_np, use_random_connections)
    
    print(f'{network} stats - auc mean: {np.mean(auc_scores):.3f}, f1 mean: {np.mean(f1_scores):.3f}, precision mean: {np.mean(precision_scores):.3f}, recall mean: {np.mean(recall_scores):.3f}')
    wilcoxon_output = wilcoxon(auc_scores, [0.5] * len(auc_scores), alternative='greater')
    print(f'result sig: {wilcoxon_output.pvalue}')

whole-brain stats - auc mean: 0.495, f1 mean: 0.315, precision mean: 0.296, recall mean: 0.340
result sig: 0.8441028594970703
pallidal-sensorimotor stats - auc mean: 0.427, f1 mean: 0.313, precision mean: 0.330, recall mean: 0.322
result sig: 0.9999475479125977
cerebellar-motor stats - auc mean: 0.519, f1 mean: 0.323, precision mean: 0.340, recall mean: 0.355
result sig: 0.0291290283203125
cerebellar-prefrontal stats - auc mean: 0.628, f1 mean: 0.402, precision mean: 0.410, recall mean: 0.414
result sig: 9.5367431640625e-07
top-cerebellar-prefrontal stats - auc mean: 0.735, f1 mean: 0.462, precision mean: 0.530, recall mean: 0.571
result sig: 9.5367431640625e-07
top-whole-brain stats - auc mean: 0.742, f1 mean: 0.478, precision mean: 0.488, recall mean: 0.605
result sig: 9.5367431640625e-07
